# Get Solr cluster status

In [63]:
import requests
import json

response = requests.get('http://solr1:8983/solr/admin/collections?action=clusterstatus&wt=json')
print(json.dumps(response.json(), indent=2))

{
  "cluster": {
    "collections": {
      "nexustiles": {
        "replicationFactor": "1", 
        "configName": "nexustiles", 
        "maxShardsPerNode": "1", 
        "shards": {
          "shard2": {
            "range": "d5550000-2aa9ffff", 
            "state": "active", 
            "replicas": {
              "core_node3": {
                "core": "nexustiles_shard2_replica1", 
                "state": "active", 
                "base_url": "http://172.20.0.12:8983/solr", 
                "leader": "true", 
                "node_name": "172.20.0.12:8983_solr"
              }
            }
          }, 
          "shard3": {
            "range": "2aaa0000-7fffffff", 
            "state": "active", 
            "replicas": {
              "core_node2": {
                "core": "nexustiles_shard3_replica1", 
                "state": "active", 
                "base_url": "http://172.20.0.13:8983/solr", 
                "leader": "true", 
                "node_name": "172.20.

# Get number of documents in Solr

In [64]:
import requests

response = requests.get('http://solr1:8983/solr/nexustiles/select?q=*:*&wt=json&rows=0')
data = response.json()
print("Number of documents found: %d" % data['response']['numFound'])

Number of documents found: 0


# Get Cassandra cluster status

In [43]:
from cassandra.cluster import Cluster

cluster = Cluster(['cassandra1', 'cassandra2', 'cassandra3'])
session = cluster.connect(keyspace="nexustiles")
print("Connected to Cassandra cluster successfully.")
results = session.execute("select count(*) from sea_surface_temp")
print("Number of rows found: %d" % results[0].count)
cluster.shutdown()

Connected to Cassandra cluster successfully.
Number of rows found: 0


# Get Mesos cluster status

In [ ]:
import requests
import json

response = requests.get('http://mesos-master:5050/state.json')
print(json.dumps(response.json()['slaves'], indent=2))

# Get Spark cluster job status

In [ ]:
import requests

response = requests.get('http://nexus-webapp:4040/api/v1/applications')
appId = response.json()[0]['id']
response = requests.get("http://nexus-webapp:4040/api/v1/applications/%s/jobs" % appId)
for job in response.json():
    print(job['name'])
    print('\t' + job['status'])

# Get Nexus dataset list

In [ ]:
import requests
from datetime import datetime

response = requests.get('http://nexus-webapp:8083/list')
data = response.json()
for dataset in data:
    print(dataset['shortName'])
    print('\t' + datetime.utcfromtimestamp(dataset['start']/1000).strftime('%Y-%m-%dT%H:%M:%SZ'))
    print('\t' + datetime.utcfromtimestamp(dataset['end']/1000).strftime('%Y-%m-%dT%H:%M:%SZ'))

# Get listing of granules and tile count

In [ ]:
import requests

dataset = 'AVHRR_OI_L4_GHRSST_NCEI'
year = 2016

response = requests.get("http://solr1:8983/solr/nexustiles/query?q=granule_s:%d*&rows=0&fq=dataset_s:%s&facet.field=granule_s&facet=true&facet.mincount=1&facet.limit=-1&facet.sort=index" % (year, dataset))
data = response.json()
for k in data['facet_counts']["facet_fields"]['granule_s']:
    print(k)

# Get number of granules ingested

In [ ]:
import requests

dataset = 'AVHRR_OI_L4_GHRSST_NCEI'
year = 2016

response = requests.get("http://solr1:8983/solr/nexustiles/query?q=granule_s:%d*&json.facet={granule_s:'unique(granule_s)'}&rows=0&fq=dataset_s:%s" % (year, dataset))
data = response.json()
print("Number of granules for %s : %d" % (dataset, data['facets']['granule_s']))